## Simple CNN with tensorflow

In [2]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
from tqdm import tqdm_notebook as tqdm

%load_ext autoreload
%autoreload
%config Completer.use_jedi=False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
tf.executing_eagerly()

True

## Read in data

Using MNIST

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [5]:
fac = 255 * 0.99 + 0.01

X_train = X_train / fac
X_test = X_test / fac
X_train = X_train.astype(np.float32)    
X_test = X_test.astype(np.float32)

In [6]:
X_train.shape, y_train.shape

((60000, 28, 28, 1), (60000,))

In [7]:
X_test.shape, y_test.shape

((10000, 28, 28, 1), (10000,))

In [29]:
x = np.ones((60_000, 784))

Convert numpy tensors to tensorflow tensors and create batches

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

Customizable training loop

In [25]:
# Define cnn
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer="he_uniform", activation="relu", input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_uniform"),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_uniform"),
#     tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer="he_uniform"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define loss function
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_epoch_loss_avg = tf.keras.metrics.Mean()
test_epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [26]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               692352    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
________________________________________________

In [27]:
@tf.function
def train_loop(x, y):

    # Calculate gradients
    with tf.GradientTape() as t:
        # training=training is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(x, training=True)
        loss = loss_func(y, predictions)

    grads = t.gradient(loss, model.trainable_variables)

    # Optimize the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Track progress
    epoch_loss_avg(loss)

    # Compare predicted label to actual
    epoch_accuracy.update_state(y, predictions)
    
#     return loss, predictions

In [28]:
%%time
train_loss_results = []
train_accuracy_results = []
epochs = 10
n_batches = len(list(train_dataset))

for epoch in tqdm(np.arange(epochs)):
    
    for x, y in tqdm(train_dataset, total=n_batches, leave=False):
        train_loop(x, y)

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    # Test
    for (x_valid, y_valid) in test_dataset:
        preds_test = model(x_valid, training=False)
        test_loss = loss_func(y_valid, preds_test)
        test_epoch_loss_avg(test_loss)
        test_epoch_accuracy.update_state(y_valid, preds_test)
    
    print(f"Epoch {epoch:03d}: train_loss: {epoch_loss_avg.result():.3f}, "
          f"test_loss: {test_epoch_loss_avg.result():.3f} "
          f"Accuracy: {epoch_accuracy.result():.3f}%"
          f"Test accuracy={test_epoch_accuracy.result():.3f}"
    )
    
    # Clear the current state of the metrics
    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()
    test_epoch_loss_avg.reset_states()
    test_epoch_accuracy.reset_states()
    # valid_loss.reset_states(), valid_acc.reset_states()
    
    

Epoch 000: train_loss: 1.590, test_loss: 1.506 Accuracy: 0.880%Test accuracy=0.958


Epoch 001: train_loss: 1.515, test_loss: 1.489 Accuracy: 0.951%Test accuracy=0.974


Epoch 002: train_loss: 1.500, test_loss: 1.485 Accuracy: 0.965%Test accuracy=0.977


Epoch 003: train_loss: 1.494, test_loss: 1.482 Accuracy: 0.970%Test accuracy=0.980


Epoch 004: train_loss: 1.489, test_loss: 1.479 Accuracy: 0.975%Test accuracy=0.983


Epoch 005: train_loss: 1.485, test_loss: 1.478 Accuracy: 0.978%Test accuracy=0.984


Epoch 006: train_loss: 1.484, test_loss: 1.477 Accuracy: 0.979%Test accuracy=0.985


Epoch 007: train_loss: 1.482, test_loss: 1.478 Accuracy: 0.981%Test accuracy=0.984


Epoch 008: train_loss: 1.481, test_loss: 1.478 Accuracy: 0.982%Test accuracy=0.983


Epoch 009: train_loss: 1.479, test_loss: 1.477 Accuracy: 0.983%Test accuracy=0.985

CPU times: user 8min 37s, sys: 4min 59s, total: 13min 36s
Wall time: 1min 56s
